<a href="https://colab.research.google.com/github/Adaline-K/git4-collaboration-sourcetree/blob/master/tensortest7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#학습이 잘 되는 모델구현(Bach Normalization)
배치 정규화란 활성화함수의 ㅣ활성화값 또는 출력값을 정규화(정규분포)하는 작업


1.   학습 속도가 개선됨(학습률을 높게 설정할 수 있기 때문)
2.   가중치 초깃값 선택의 의존성이 적어짐(학습을 할때마다 출력값을 정규화하기 때문)
3.   과적합(Overfitting)위험을 줄일 수 있음
4.   Gradient Vanishing 문제해결



In [2]:
#라이브러리 사용
import tensorflow as tf
import pandas as pd

In [3]:
#보스턴 집값 예측
#과거 데이터 준비
파일경로 = 'https://raw.githubusercontent.com/blackdew/tensorflow1/master/csv/boston.csv'
보스턴 = pd.read_csv(파일경로)
print(보스턴.columns)

#종속, 독립변수
독립 = 보스턴[['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'b', 'lstat']]
종속 = 보스턴[['medv']]
print(독립.shape, 종속.shape)

Index(['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'b', 'lstat', 'medv'],
      dtype='object')
(506, 13) (506, 1)


In [5]:
#모델구조만들기
X = tf.keras.layers.Input(shape=[13])

H = tf.keras.layers.Dense(8, activation='swish')(X)
H = tf.keras.layers.Dense(8, activation='swish')(H)
H = tf.keras.layers.Dense(8, activation='swish')(H)

Y = tf.keras.layers.Dense(1)(H)
model = tf.keras.models.Model(X,Y)
model.compile(loss='mse')

In [8]:
#모델의 구조를 BachNormalization layer를 사용하여 만든다
X = tf.keras.layers.Input(shape=[13])

H = tf.keras.layers.Dense(8)(X)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

H = tf.keras.layers.Dense(8)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

H = tf.keras.layers.Dense(8)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

Y = tf.keras.layers.Dense(1)(H)
model = tf.keras.models.Model(X,Y)
model.compile(loss='mse')

In [9]:
#모델구조확인
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 112       
_________________________________________________________________
batch_normalization_3 (Batch (None, 8)                 32        
_________________________________________________________________
activation_3 (Activation)    (None, 8)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 8)                 72        
_________________________________________________________________
batch_normalization_4 (Batch (None, 8)                 32        
_________________________________________________________________
activation_4 (Activation)    (None, 8)                 0   

In [12]:
#데이터로 모델학습시키기(FIT)
model.fit(독립, 종속, epochs=1000)


Epoch 1/1000
16/16 [==============================] - 0s 2ms/step - loss: 324.5288
Epoch 2/1000
16/16 [==============================] - 0s 1ms/step - loss: 316.5475
Epoch 3/1000
16/16 [==============================] - 0s 1ms/step - loss: 304.8974
Epoch 4/1000
16/16 [==============================] - 0s 1ms/step - loss: 296.3157
Epoch 5/1000
16/16 [==============================] - 0s 1ms/step - loss: 287.4511
Epoch 6/1000
16/16 [==============================] - 0s 1ms/step - loss: 281.0992
Epoch 7/1000
16/16 [==============================] - 0s 2ms/step - loss: 270.8647
Epoch 8/1000
16/16 [==============================] - 0s 2ms/step - loss: 261.7698
Epoch 9/1000
16/16 [==============================] - 0s 1ms/step - loss: 254.3975
Epoch 10/1000
16/16 [==============================] - 0s 1ms/step - loss: 247.0628
Epoch 11/1000
16/16 [==============================] - 0s 2ms/step - loss: 237.4933
Epoch 12/1000
16/16 [==============================] - 0s 1ms/step - loss: 228.5563
E

In [13]:
#모델이용
print(model.predict(독립[0:5]))
print(종속[0:5])


[[23.828993]
 [21.726341]
 [28.042112]
 [28.85323 ]
 [29.001196]]
   medv
0  24.0
1  21.6
2  34.7
3  33.4
4  36.2


In [14]:
#아이리스 품종 분류
#과거의 데이터 준비
파일경로 = 'https://raw.githubusercontent.com/blackdew/tensorflow1/master/csv/iris.csv'
아이리스 = pd.read_csv(파일경로)

#원핫인코딩
아이리스 = pd.get_dummies(아이리스)
print(아이리스.columns)
#독립, 종속변수
독립 = 아이리스[['꽃잎길이', '꽃잎폭', '꽃받침길이', '꽃받침폭']]
종속 = 아이리스[['품종_setosa', '품종_versicolor',
       '품종_virginica']]
print(독립.shape, 종속.shape)

Index(['꽃잎길이', '꽃잎폭', '꽃받침길이', '꽃받침폭', '품종_setosa', '품종_versicolor',
       '품종_virginica'],
      dtype='object')
(150, 4) (150, 3)


In [15]:
#모델구조 만들기
X = tf.keras.layers.Input(shape=[4])
H = tf.keras.layers.Dense(8, activation='swish')(X)
H = tf.keras.layers.Dense(8, activation='swish')(H)
H = tf.keras.layers.Dense(8, activation='swish')(H)
Y = tf.keras.layers.Dense(3, activation='softmax')(H)
model = tf.keras.models.Model(X,Y)
model.compile(loss='categorical_crossentropy', metrics='accuracy')

In [18]:
#모델구조를 BachNormalizaiton layer를 사용하여 만든다.
X = tf.keras.layers.Input(shape=[4])

H = tf.keras.layers.Dense(8)(X)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

H = tf.keras.layers.Dense(8)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

H = tf.keras.layers.Dense(8)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

Y = tf.keras.layers.Dense(3, activation='softmax')(H)
model = tf.keras.models.Model(X,Y)
model.compile(loss='categorical_crossentropy', metrics='accuracy')

In [19]:
#모델구조 확인
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_23 (Dense)             (None, 8)                 40        
_________________________________________________________________
batch_normalization_6 (Batch (None, 8)                 32        
_________________________________________________________________
activation_9 (Activation)    (None, 8)                 0         
_________________________________________________________________
dense_24 (Dense)             (None, 8)                 72        
_________________________________________________________________
batch_normalization_7 (Batch (None, 8)                 32        
_________________________________________________________________
activation_10 (Activation)   (None, 8)                 0   

In [20]:
#데이터로 모델 학습시키기(FIT)
model.fit(독립, 종속, epochs=100)

Epoch 1/100
5/5 [==============================] - 1s 2ms/step - loss: 1.3311 - accuracy: 0.2679
Epoch 2/100
5/5 [==============================] - 0s 2ms/step - loss: 1.1942 - accuracy: 0.3395
Epoch 3/100
5/5 [==============================] - 0s 2ms/step - loss: 1.0935 - accuracy: 0.4220
Epoch 4/100
5/5 [==============================] - 0s 2ms/step - loss: 1.0456 - accuracy: 0.4094
Epoch 5/100
5/5 [==============================] - 0s 2ms/step - loss: 0.9077 - accuracy: 0.5240
Epoch 6/100
5/5 [==============================] - 0s 2ms/step - loss: 0.8236 - accuracy: 0.5899
Epoch 7/100
5/5 [==============================] - 0s 2ms/step - loss: 0.8607 - accuracy: 0.5444
Epoch 8/100
5/5 [==============================] - 0s 2ms/step - loss: 0.7638 - accuracy: 0.6393
Epoch 9/100
5/5 [==============================] - 0s 2ms/step - loss: 0.7726 - accuracy: 0.6603
Epoch 10/100
5/5 [==============================] - 0s 2ms/step - loss: 0.7627 - accuracy: 0.6710
Epoch 11/100
5/5 [===========

In [21]:
#모델이용
print(model.predict(독립[0:5]))
print(종속[0:5])

[[9.9989283e-01 6.9420312e-05 3.7819482e-05]
 [9.9935740e-01 3.4403679e-04 2.9849299e-04]
 [9.9989533e-01 5.6381003e-05 4.8270500e-05]
 [9.9963439e-01 2.0437570e-04 1.6127073e-04]
 [9.9994290e-01 3.8094098e-05 1.9005596e-05]]
   품종_setosa  품종_versicolor  품종_virginica
0          1              0             0
1          1              0             0
2          1              0             0
3          1              0             0
4          1              0             0
